In [117]:
import pandas as pd
import json
from py2neo import Graph, Node, Relationship

graph  = Graph("http://localhost:7474/db/data/",user='neo4j',password='ingamc03')

In [236]:
dftweetslabeled = pd.read_csv('tweets_analyzed.csv', encoding='utf-8')
dftweetslabeled.head(2)
dftweetslabeled=dftweetslabeled.rename(columns = {'Unnamed: 0.1':'id'})
dftweetslabeled.head(1)

,Unnamed: 0,id,userId,geo,coordinates,timestamp,tweet,source,isretweeted,lang,lat,lon,polarity
0,0,800355989086203905,799032032017993733,NaN,NaN,1479654712350,rt AT_USER el edificio (no okupado) que posee ...,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",True,es,null,null,P


In [325]:
class user_class:
    def __init__(self, name, description, followers, following):
        self.name = name
        self.desc = description
        self.followers = followers
        self.following = following
    
class tweet_class:
    def __init__(self, name, autor, text, source, retweeted, publisher, polarity):
        self.identifier = name
        self.autor = autor
        self.text = text
        self.source = source
        self.retweeted = retweeted
        self.publisher = publisher
        self.polarity = polarity
        
def create_node_user(user,graph):
    try:
        user_node = Node("User", id=str(user.name), desc=user.desc, followers=user.followers, following=user.following)
        graph.merge(user_node)
    except Exception as e:
        print e.args,':','creating user node graph'
        
    return user_node
    
def create_relationship(tweet,graph):
    #publish relationship when is retwweted from
    #autor the original tweeter user
    #publisher is who publish the RT
    try:   
        tweet_node = Node("Tweet", id=str(tweet.identifier), source=tweet.source, text=tweet.text, retweeted=str(tweet.retweeted), 
                          polarity= tweet.polarity)
        graph.merge(tweet_node)
    except Exception as e:
        print e.args,':','creating tweet node graph'
    
    if (tweet.retweeted):
        pub = create_node_user(tweet.publisher,graph)
        graph.create(rel(tweet_node, "RT_BY", pub))
    else:
        au = create_node_user(tweet.autor,graph)
        graph.create(rel(tweet_node, "CREATED_BY", au))    

In [323]:
def processUser(decoded,df):
    try:
        userId = decoded['user']['id']
        name = decoded['user']['screen_name']
        description = decoded['user']['description']
        url = decoded['user']['url']
        followers = decoded['user']['followers_count']
        lang = decoded['user']['lang']
        profile_image = decoded['user']['profile_image_url']
        following = decoded['user']['following']
        print 'NAME:'+ name
        print 'DESC:'+ description
        print 'FOLLOWERS:'+ str(followers)
        print 'FOLLOWING:'+ str(following)
        #df.loc[userId] = [name,description,url,followers,lang,profile_image,following,followers]
        us = user_class(name, description, str(followers), str(following))
    except Exception as a:
        print a.args,':','error processing User......'
    return us

def processAutor(decoded):
    try:
        #userId = decoded['retweeted_status']['user']['screen_name']
        name = decoded['retweeted_status']['user']['screen_name']
        description = decoded['retweeted_status']['user']['description']
        #url = decoded['retweeted_status']['user']['screen_name']
        followers = decoded['retweeted_status']['user']['followers_count']
        #lang = decoded['retweeted_status']['user']['screen_name']
        #profile_image = decoded['retweeted_status']['user']['screen_name']
        following = decoded['retweeted_status']['user']['following']
        us = user_class(name, description, followers, following)
    except Exception as a:
        print a.args,':','error processing Author......'
    return us

#Proceso de Tweets
def processTweet(decoded,df):
    try:
        identifier = decoded['id']
        #print 'IDENTIFICADOR..............:' + identifier
        publisher = decoded['user']['screen_name']
        geo = decoded['geo']
        coordinates = decoded['coordinates']
        timestamp = decoded['timestamp_ms']
        text = decoded['text']
        text = text.lower()
        source = decoded['source']
        isretweeted = False 
        autor=''
        autor_class = ''
        if 'retweeted_status' in decoded:
            if decoded['retweeted_status'] is not 'null':
                isretweeted = True
                autor = decoded['retweeted_status']['user']['screen_name']
                autor_class = processAutor(decoded)
        lang = decoded['lang']
        lat = 'null'
        lon = 'null'
        polarity = 'null'
        print 'Procesaudo el publisher.....'
        publisher_class = processUser(decoded,df)
        print 'Procesado....'
        df.loc[identifier] = [publisher,geo,coordinates,timestamp,text,source,isretweeted,lang,lat,lon, polarity] 
        tw = tweet_class(identifier, autor, text, source, isretweeted, publisher, polarity)
    except Exception as a:
        print a.args,':','error processing Tweet'
    
    return tw


In [326]:
file = open('C:\\Users\\Alberto\\PycharmProjects\\twitterExtraction\\CarmenaTweets_Test.txt','r')
decoded = ''
counter = 0
for line in file: 
    try:
        decoded = json.loads(line)
    except Exception as e:
        print e,':',decoded['id'] 
        
    try:
        user = processUser(decoded, dfusers)
    except Exception as a:
        print a.args,':','error processing users'
        
    try:
        tw = processTweet(decoded,dftweets)
    except Exception as a:
        print a.args,':','error processing tweet'
        
    try:
        create_relationship(tw,graph)
    except Exception as a:
        print a.args,':','creating node and relationship'
        
    

NAME:SenioritoLuShin
DESC:ateo. nihilista. republicano (no monarquico), un poco feminista
FOLLOWERS:87
FOLLOWING:None
Procesaudo el publisher.....
NAME:SenioritoLuShin
DESC:ateo. nihilista. republicano (no monarquico), un poco feminista
FOLLOWERS:87
FOLLOWING:None
Procesado....
("'unicode' object has no attribute 'name'",) : creating node and relationship
NAME:biferato
DESC:Podemos cambiar España, está en nuestras manos
FOLLOWERS:71
FOLLOWING:None
Procesaudo el publisher.....
NAME:biferato
DESC:Podemos cambiar España, está en nuestras manos
FOLLOWERS:71
FOLLOWING:None
Procesado....
("'unicode' object has no attribute 'name'",) : creating node and relationship
NAME:patxi7045
("cannot concatenate 'str' and 'NoneType' objects",) : error processing User......
("local variable 'us' referenced before assignment",) : error processing users
Procesaudo el publisher.....
NAME:patxi7045
("cannot concatenate 'str' and 'NoneType' objects",) : error processing User......
("local variable 'us' refere

In [17]:
dfusers = pd.DataFrame(columns=['name','description','url','followers','lang','profile_image','following','followers'])
dftweets = pd.DataFrame(columns=['userId','geo','coordinates','timestamp','tweet','source','isretweeted','lang','lat','lon','polarity'])

In [5]:
dfusers.head(2)

,Unnamed: 0,name,location,url,followers,lang,profile_image,following
0,799032032017993733,GRUPO METRO MENDOZA,"Mendoza, Argentina",http://www.grupometromza.com,24.0,es,http://pbs.twimg.com/profile_images/7990356908...,NaN
1,2326149346,Pepe López,NaN,NaN,329.0,es,http://pbs.twimg.com/profile_images/4310926238...,NaN
